# Big Red Soundscape
Notebook Playground 2  
_by Ahmed Sultan and Carly Hu, for Cornell Data Journal_

In [1]:
import json
import math

import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# update access keys here for Spotify API here :) 
client_id = 'f7661e34c0924317a8b928c0f977e6a7'
client_secret = 'd068ec3f0a98490e9c32be820916e934'

# sp is how we interface with the Spotify API
# read the docs here: https://spotipy.readthedocs.io/en/2.22.1/
sp = spotipy.Spotify(client_credentials_manager=\
                     SpotifyClientCredentials(client_id, client_secret))

<h3> Look at your own stats! </h3>

Feel free to put in your own spotify username in `selected_user` below, and if you run every cell, by the end of the notebook you will be able to see the information & statistics for __all of the songs that are on your public playlists__ (with duplicates removed). Have fun!

In [3]:
# Big Data, Machine Learning...
# https://www.youtube.com/watch?v=bqqCTC9nQDY

# let's aggregate an entire user's public library of songs
playlists = []
LIMIT = 20
selected_user = 'kaybae.'

query = sp.user_playlists(selected_user, limit=LIMIT)

n = 0

while len(query['items']) != 0:
    n += LIMIT
    for i in query['items']:
        playlists.append({'title': i['name'],'id': i['id'], 'length': i['tracks']['total']})
    if LIMIT == len(query['items']):
        query = sp.user_playlists(selected_user, limit=LIMIT, offset=n)
    else:
        break
        
# there's definitely a cleaner way to do this that we'll figure out
# im definitely lazy

In [4]:
playlists

[{'title': 'name 4 brothers or gtfo‼️',
  'id': '2zl5Uv8NGl6Q3g5dc6h6ov',
  'length': 255},
 {'title': 'tb to the good old days',
  'id': '4UlstL86DQfMRsS7GHIki0',
  'length': 458},
 {'title': 'sand in ma cheeks', 'id': '6rEePsA1H2OgSfFoDmTOnT', 'length': 427},
 {'title': 'top down w nowhere to go ',
  'id': '3xjLof9WnGgR1OpSUBnXiw',
  'length': 186},
 {'title': 'mindfuck me harder', 'id': '7uw8pNEERGXrt5Vw94mzsJ', 'length': 91},
 {'title': 'i just wanna feel smth',
  'id': '0GCkzwiNzJlh1lH2sR30In',
  'length': 231},
 {'title': 'with u', 'id': '51KgyolANfrXTVsSSjuk0D', 'length': 298},
 {'title': 'broc', 'id': '7s314m4v9udXpzfM2uILUP', 'length': 115},
 {'title': 'screaming', 'id': '52h6DmzwNslFlYAKxUo7cA', 'length': 63},
 {'title': 'sometimes there is no why',
  'id': '4gVaC4rRqXAB1NTkx7GHTu',
  'length': 426},
 {'title': 'more juice pls', 'id': '1ICgNreX5Eev9bPJUFu1LI', 'length': 703},
 {'title': 'sweatbox', 'id': '0PsINHKMfS0hH0b5vjSOZW', 'length': 79},
 {'title': 'violent vibrations 

In [5]:
# number of public playlists
print(len(playlists))

34


In [6]:
# making sure we can get all songs in a playlist rather than just the first 100
# https://stackoverflow.com/questions/55690063/is-there-a-method-to-retrieve-all-tracks-from-a-playlist-using-spotipy 

def user_playlist_tracks_full(user, playlist_id=None, fields=None):
    """ Get full details of the tracks of a playlist owned by a user.
        https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlists-tracks/

        Parameters:
            - user - the id of the user
            - playlist_id - the id of the playlist
            - fields - which fields to return
    """

    # first run through also retrieves total no of songs in library
    response = sp.user_playlist_tracks(user, playlist_id, fields=None, limit=100)
    results = response["items"]

    # subsequently runs until it hits the user-defined limit or has read all songs in the library
    while len(results) < response["total"]:
        response = sp.user_playlist_tracks(
            user, playlist_id, fields=fields, limit=100, offset=len(results)
        )
        results.extend(response["items"])
    
    return results
        

In [7]:
# now, let's go through each playlist
# this will basically be the same as before

songs = []

for p in playlists:
    
    results = sp.playlist(p['id'])
    
    extended_results = user_playlist_tracks_full(user = selected_user, playlist_id = results['id']) # more than 100
    
    for r in extended_results:
        try:
            songs.append(r['track']['id']) 
        except:
            pass

print(len(songs)) # this is the total number of songs on your public playlists

6091


In [8]:
# helper function
def get_tracks_full(songs):
    results = []
    while len(results) < len(songs):
        response = (sp.tracks(songs[len(results):len(results) + 50]))['tracks']
        results.extend(response)
    return results

In [9]:
# this cell is creating a dataframe of your song information: artist, album, popularity, etc.

songs_meta = get_tracks_full(songs)

song_info = {'id': [], 'title': [], 'artist': [], 'album': [], 
             'explicit': [], 'popularity': []}
    
for i in songs_meta:
    # append id to id column
    song_info['id'].append(i['id'])
    
    # append song title to title column
    song_info['title'].append(i['name'])
    
    # append artists to artist column
    s = ', '
    artists = s.join([name['name'] for name \
                     in i['artists']])
    song_info['artist'].append(artists)
    
    # append album title to album column
    song_info['album'].append(i['album']['name'])
    
    # append explicit info to explicit column
    song_info['explicit'].append(i['explicit'])
    
    # append popularity info to popularity column
    song_info['popularity'].append(i['popularity'])
    
# convert song_info dictionary -> Pandas DataFrame
song_info_df = pd.DataFrame.from_dict(song_info)

song_info_df = song_info_df.drop_duplicates(subset=['id'])
song_info_df = song_info_df.dropna()
print(song_info_df.columns)
print(song_info_df.shape)

Index(['id', 'title', 'artist', 'album', 'explicit', 'popularity'], dtype='object')
(4410, 6)


In [10]:
#helper function 

def get_audio_features_full(songs):
    results = []
    while len(results) < len(songs):
        response = sp.audio_features(songs[len(results):len(results) + 100])
        response = list(filter(None, response))
        results.extend(response)
    return results

In [11]:
# this cell is creating a dataframe of your song features: danceability, energy, loudness, etc

features_df = pd.DataFrame.from_dict(get_audio_features_full(songs))
features_df = features_df.drop_duplicates(subset=['id'])
features_df = features_df.dropna()
print(features_df.columns)
print(features_df.shape)

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')
(4409, 18)


In [12]:
# combining the two cleaned dataframes....
df = song_info_df.merge(features_df)
print(df.shape)
df

(4409, 23)


,id,title,artist,album,explicit,popularity,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,5VDHMi7own9vDZjiwDES1e,Wildest Dreams - R3hab Remix,"Taylor Swift, R3HAB",Wildest Dreams (R3hab Remix),False,55,0.584,0.809,8,-4.352,...,0.0000,0.0888,0.247,130.104,audio_features,spotify:track:5VDHMi7own9vDZjiwDES1e,https://api.spotify.com/v1/tracks/5VDHMi7own9v...,https://api.spotify.com/v1/audio-analysis/5VDH...,198107,4
1,0ntQJM78wzOLVeCUAW7Y45,Sex on Fire,Kings of Leon,Only By The Night,False,83,0.542,0.905,9,-5.653,...,0.0104,0.1360,0.374,153.398,audio_features,spotify:track:0ntQJM78wzOLVeCUAW7Y45,https://api.spotify.com/v1/tracks/0ntQJM78wzOL...,https://api.spotify.com/v1/audio-analysis/0ntQ...,203347,4
2,0Gc1AVYqYOR522Nvz68dsT,The Hum,"Dimitri Vegas & Like Mike, Ummet Ozcan",The Hum,False,53,0.728,0.947,11,-3.103,...,0.8150,0.3670,0.209,130.007,audio_features,spotify:track:0Gc1AVYqYOR522Nvz68dsT,https://api.spotify.com/v1/tracks/0Gc1AVYqYOR5...,https://api.spotify.com/v1/audio-analysis/0Gc1...,211000,4
3,6hEJvWfNOvTUhLuV1Or1of,Whistle (feat. Bertie Scott),"Laidback Luke, Tribbs, Bertie Scott",Whistle (feat. Bertie Scott),False,66,0.832,0.922,0,-3.866,...,0.0274,0.1090,0.716,127.999,audio_features,spotify:track:6hEJvWfNOvTUhLuV1Or1of,https://api.spotify.com/v1/tracks/6hEJvWfNOvTU...,https://api.spotify.com/v1/audio-analysis/6hEJ...,140156,4
4,62hxFc3yOEyauOuuqbKqxt,Teenage Dirtbag,"Honey Gee, Crystal Rock, Moestwanted",Teenage Dirtbag,True,44,0.617,0.527,4,-6.953,...,0.0000,0.1610,0.268,120.049,audio_features,spotify:track:62hxFc3yOEyauOuuqbKqxt,https://api.spotify.com/v1/tracks/62hxFc3yOEya...,https://api.spotify.com/v1/audio-analysis/62hx...,160000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4404,7Lbxvt42JsNSKVCaN63qhD,Wonderful Christmastime [Edited Version] - Rem...,Paul McCartney,McCartney II (Special Edition),False,26,0.774,0.511,11,-8.914,...,0.0000,0.0672,0.771,94.736,audio_features,spotify:track:7Lbxvt42JsNSKVCaN63qhD,https://api.spotify.com/v1/tracks/7Lbxvt42JsNS...,https://api.spotify.com/v1/audio-analysis/7Lbx...,227653,4
4405,6tjituizSxwSmBB5vtgHZE,Holly Jolly Christmas,Michael Bublé,Christmas,False,47,0.647,0.465,7,-8.308,...,0.0000,0.0882,0.699,151.251,audio_features,spotify:track:6tjituizSxwSmBB5vtgHZE,https://api.spotify.com/v1/tracks/6tjituizSxwS...,https://api.spotify.com/v1/audio-analysis/6tji...,119787,4
4406,3hK0tQgnfjXwaCd0oHV0u2,K For Christmas,Lil Mosey,K For Christmas,True,38,0.707,0.530,2,-9.471,...,0.0044,0.1120,0.471,189.008,audio_features,spotify:track:3hK0tQgnfjXwaCd0oHV0u2,https://api.spotify.com/v1/tracks/3hK0tQgnfjXw...,https://api.spotify.com/v1/audio-analysis/3hK0...,102162,4
4407,1ADjWm8QNhgNV8yCNNgQ1T,Santa Tell Me,Ariana Grande,Santa Tell Me,False,0,0.464,0.630,7,-7.337,...,0.0000,0.2950,0.527,132.960,audio_features,spotify:track:1ADjWm8QNhgNV8yCNNgQ1T,https://api.spotify.com/v1/tracks/1ADjWm8QNhgN...,https://api.spotify.com/v1/audio-analysis/1ADj...,204093,5


In [13]:
features = ['popularity','danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

feature_averages = []
for feature in features:
    avg =  df[feature].mean()
    feature_averages.append(avg)

print(feature_averages)

[55.77114992061692, 0.6647525515990023, 0.6218481288274003, 5.227489226581992, -6.7952211385801755, 0.5856203220684962, 0.11514121115899308, 0.2367677097527781, 0.033611861295078274, 0.17849961442503992, 0.4773371286005891, 121.07640757541397]
